In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [3]:
df = pd.read_csv('data/Bank-Customer-Churn-Prediction.csv')
df.pop('churn')

columns = df.columns.values
random = random.randint(0, len(columns))

features = df.values[random]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'customer_id': 15701354,
 'credit_score': 699,
 'country': 'France',
 'gender': 'Female',
 'age': 39,
 'tenure': 1,
 'balance': 0.0,
 'products_number': 2,
 'credit_card': 0,
 'active_member': 0,
 'estimated_salary': 93826.63}

In [4]:
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )

def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, 'utf-8')]
        ),
    )

In [6]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result,
    })

In [10]:
def predict(inputs):
    response = requests.post(
        'https://bank-cust-churn-production.up.railway.app/v1/models/bank-churn-model:predict',
        data=prepare_json(inputs)
    )

    predict = response.json()['predictions'][0][0]
    return predict

In [11]:
predict = predict(inputs)

print(f'{inputs}\n')
print(f'Churn' if predict > 0.6 else f'No Churn')

{'customer_id': 15701354, 'credit_score': 699, 'country': 'France', 'gender': 'Female', 'age': 39, 'tenure': 1, 'balance': 0.0, 'products_number': 2, 'credit_card': 0, 'active_member': 0, 'estimated_salary': 93826.63}

No Churn
